<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/profile_opset/div/div.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#!pip install onnx onnxruntime


In [2]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name = "X"
log_output_name = "LogOutput"

#-------------------------------------------------------------------------------
# 2-rank input tensor
#-------------------------------------------------------------------------------

# Create 2-rank input tensor
input_tensor = onnx.helper.make_tensor_value_info(input_name,onnx.TensorProto.FLOAT,[None, None])

# Create output tensor (final result after log operation)
log_output = onnx.helper.make_tensor_value_info(log_output_name,onnx.TensorProto.FLOAT,[None, None])

# Define Log node: Y = Log(X)
log_node = onnx.helper.make_node("Log",[input_name],[log_output_name])

# Create the ONNX graph
graph = onnx.helper.make_graph(
    nodes=[log_node],
    name="Log",
    inputs=[input_tensor],
    outputs=[log_output]
)

# Create the ONNX model (Log is available in opset 13)
model = onnx.helper.make_model(graph,opset_imports=[onnx.helper.make_opsetid("", 13)]) # Explicitly set opset to 13
onnx.checker.check_model(model)

# Save the model
onnx.save(model, "log.onnx")

# Load and run the model using ONNX Runtime
session = ort.InferenceSession("log.onnx")

def do_log(x):
    # Run inference
    output = session.run(None, {input_name: x})

    x_f = (np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', ''))
    o_f = (np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', ''))

    # Display results
    print(f"X={x_f}")
    print(f"Result: log(X)={o_f}")

np.set_printoptions(precision=None, floatmode='fixed')


#--------------------------------------------------------------------
# Nominal cases
#--------------------------------------------------------------------

# Case N1: 2-rank tensor (float32)
x = np.array([[1.0, 2.0],
              [3.0, 4.0]], dtype=np.float32)

do_log(x)

#--------------------------------------------------------------------
# Non nominal cases (nan, -inf, 0, negative values)
#--------------------------------------------------------------------

v = [1.0, 0.0, -1.0, float("inf"), float("-inf"), float("nan")]

for x_val in v:
    x_np = np.array([[x_val],
                     [x_val]], dtype=np.float32)
    do_log(x_np)


X=[[1.00000000,2.00000000], [3.00000000,4.00000000]]
Result: log(X)=[[0.00000000,0.69314718], [1.09861231,1.38629436]]
X=[[1.00000000], [1.00000000]]
Result: log(X)=[[0.00000000], [0.00000000]]
X=[[0.00000000], [0.00000000]]
Result: log(X)=[[-inf], [-inf]]
X=[[-1.00000000], [-1.00000000]]
Result: log(X)=[[nan], [nan]]
X=[[inf], [inf]]
Result: log(X)=[[inf], [inf]]
X=[[-inf], [-inf]]
Result: log(X)=[[nan], [nan]]
X=[[nan], [nan]]
Result: log(X)=[[nan], [nan]]
